In [103]:
import json
import requests
import time
import urllib
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import pandas as pd
import sys
import datetime as dt

In [2]:
# GDP indicator.Korea
# GDP = C + I + G + (X-M)
## 수출입동향 : 매월 1일 : 산업통상자원부
## 소비자물가동향 : 매월초순 : 통계청
## 고용동향 : 매월중순 : 통계청
## 산업활동동향 : 매월말 : 통계청
## GDP : 분기말의 다음월말 : 한국은행

In [209]:
# 단위 %
iaip = pd.read_excel("./생산자지수.xls")
iaip = iaip.iloc[1:-2]
iaip = iaip.T.reset_index(drop = True)
# col name 변경
new = iaip.iloc[0]
new[1] = "date"
old = [i for i in iaip.columns.values]
col_dic = {old : new for old,new in zip(old, new)}
for key, value in col_dic.items() : 
    iaip.rename({key : value}, axis = 1, inplace = True)
    
# 날짜 변경 및 인데스 설정
iaip = iaip.iloc[1:]
new_date = [i.replace("월", "") for i in iaip["date"]]
new_form = []
for date in new_date : 
    new_date = dt.datetime.strptime(date,"%Y%m").strftime("%Y-%m")
    new_form.append(new_date)
iaip["date"] = new_form
iaip = iaip.set_index("date")  

iaip

,전산업생산지수(원지수),전년동월(기)비,- 광공업,- 건설업,- 서비스업,- 공공행정
date,,,,,,
2002-01,62.6,10.8,13.9,19.8,9.8,-1.4
2002-02,58.1,6.4,-1.0,19.4,8.1,7.8
2002-03,65.9,9.3,7.0,7.9,12.0,0.2
2002-04,66.4,10.5,10.5,4.6,12.3,3.3
2002-05,67.5,8.5,8.6,2.1,10.2,5.4
...,...,...,...,...,...,...
2022-03,120.0,3.4,3.7,-5.0,3.8,5.3
2022-04,116.1,4.5,3.6,-2.4,5.0,9.2
2022-05,118.0,7.1,7.5,7.6,7.4,2.1


In [196]:
# 단위 %
cpi = pd.read_excel("./소비자물가지수.xls")
cpi = cpi.iloc[1:-2]
cpi = cpi.T.reset_index(drop = True)
# col name 변경
new = cpi.iloc[0]
new[1] = "date"
old = [i for i in cpi.columns.values]
col_dic = {old : new for old,new in zip(old, new)}
for key, value in col_dic.items() : 
    cpi.rename({key : value}, axis = 1, inplace = True)
    
# 날짜 변경 및 인데스 설정
cpi = cpi.iloc[1:]
new_date = [i.replace("월", "") for i in cpi["date"]]
new_form = []
for date in new_date : 
    new_date = dt.datetime.strptime(date,"%Y%m").strftime("%Y-%m")
    new_form.append(new_date)
cpi["date"] = new_form
cpi = cpi.set_index("date")  

cpi

,소비자물가,- 농축수산물,- 공업제품,- 집세,- 공공서비스,- 개인서비스,근원물가,생활물가
date,,,,,,,,
2002-01,2.6,8.3,-0.3,5.8,1.3,3.3,2.9,2.4
2002-02,2.6,11.2,0.2,6.1,-1.2,3.2,3.0,2.4
2002-03,2.3,5.8,0.5,6.1,-1.1,3.4,3.0,2.0
2002-04,2.5,5.4,1.1,5.9,-1.3,3.5,3.0,2.2
2002-05,3.0,9.2,1.4,6.1,-1.5,3.6,3.0,3.1
...,...,...,...,...,...,...,...,...
2022-04,4.8,1.9,7.8,2.0,0.7,4.5,3.6,5.7
2022-05,5.4,4.2,8.3,2.0,0.7,5.1,4.1,6.7
2022-06,6.0,4.8,9.3,1.9,0.7,5.8,4.4,7.4


In [203]:
# 단위 억불
xm = pd.read_excel("./무역수지.xls")
xm = xm.iloc[1:-1]
xm = xm.T.reset_index(drop = True)
# col name 변경
new = xm.iloc[0][:7]
new[1] = "date"
old = [i for i in xm.columns.values]
col_dic = {old : new for old,new in zip(old, new)}
for key, value in col_dic.items() : 
    xm.rename({key : value}, axis = 1, inplace = True)
    
# 날짜 변경 및 인데스 설정
xm = xm.iloc[1:]
new_date = [i.replace("월", "") for i in xm["date"]]
new_form = []
for date in new_date : 
    new_date = dt.datetime.strptime(date,"%Y%m").strftime("%Y-%m")
    new_form.append(new_date)
xm["date"] = new_form
xm = xm.set_index("date")       
xm

,수출,(전년동월대비 수출 증감률),수입,(전년동월대비 수입 증감률),무역수지
date,,,,,
2002-01,114,-10.0,113,-8.9,1
2002-02,110,-17.5,105,-17.8,6
2002-03,133,-6.2,120,-7.4,13
2002-04,132,8.8,126,12.0,6
2002-05,142,6.5,127,9.9,15
...,...,...,...,...,...
2022-03,638,18.8,636,27.9,2
2022-04,579,12.9,603,18.5,-25
2022-05,616,21.4,632,31.9,-16


In [144]:
# 단위 : 천명, %
employment = pd.read_excel("./고용지표.xls")
employment = employment.iloc[1:10]
employment = employment.T.reset_index(drop = True)
# col name 변경
new = employment.iloc[0][:10]
new[1] = "date"
old = [i for i in employment.columns.values]
col_dic = {old : new for old,new in zip(old, new)}
for key, value in col_dic.items() : 
    employment.rename({key : value}, axis = 1, inplace = True)
# # 날짜 변경 및 인데스 설정
employment = employment.iloc[1:]
new_date = [i.replace("월", "") for i in employment["date"]]
new_form = []
for date in new_date : 
    new_date = dt.datetime.strptime(date,"%Y%m").strftime("%Y-%m")
    new_form.append(new_date)
employment["date"] = new_form
employment = employment.set_index("date")
employment

,생산가능인구,경제활동인구,취업자,실업자,비경제활동인구,경제활동참가율(15-64세),고용률(15-64세),실업률
date,,,,,,,,
2002-01,"36,835","22,306","21,385",921,"14,529",64.5,61.7,4.1
2002-02,"36,871","22,377","21,466",912,"14,494",64.6,61.9,4.1
2002-03,"36,902","22,687","21,824",864,"14,215",65.2,62.6,3.8
2002-04,"36,934","23,147","22,363",784,"13,786",66.1,63.8,3.4
2002-05,"36,963","23,248","22,518",730,"13,715",66.3,64.1,3.1
...,...,...,...,...,...,...,...,...
2022-03,"45,219","28,627","27,754",873,"16,592",70.0,67.8,3.0
2022-04,"45,233","28,942","28,078",864,"16,291",70.6,68.4,3.0
2022-05,"45,245","29,374","28,485",889,"15,871",71.5,69.2,3.0


In [132]:
# 단위 : 십억
gdp = pd.read_excel("./KOR_GDP.xls")
gdp = gdp.iloc[1:4, 1:]
gdp = gdp.T.reset_index(drop = True)
# col name 변경
gdp.rename({1 : "date",  2: "국내총생산", 3 : "경제성장률"}, axis = 1, inplace = True)
# 분기날짜 변경
new_date = [i.replace("1/4", "-03").replace("2/4", "-06").replace("3/4", "-09").replace("4/4", "-12") for i in gdp["date"]]
gdp["date"] = new_date
gdp = gdp.set_index("date")
# 국내총생산에 있는 , 제거하고 숫자형으로 바꾸기
new_gdp = [i.replace(",", "") for i in gdp['국내총생산']]
gdp['국내총생산'] = new_gdp
#gdp['국내총생산'].astype(float)
gdp = gdp.astype({'국내총생산': 'float64', "경제성장률" : 'float64'})
gdp

,국내총생산,경제성장률
date,,
2000-03,152404.4,12.7
2000-06,160404.4,9.1
2000-09,166224.9,9.1
2000-12,172397.6,5.9
2001-03,165662.3,5.1
...,...,...
2021-03,483472.6,2.2
2021-06,514474.3,6.2
2021-09,528869.3,4.0


In [211]:
kor_macro = pd.concat([gdp, cpi, iaip, xm])
kor_macro

,국내총생산,경제성장률,소비자물가,- 농축수산물,- 공업제품,- 집세,- 공공서비스,- 개인서비스,근원물가,생활물가,...,전년동월(기)비,- 광공업,- 건설업,- 서비스업,- 공공행정,수출,(전년동월대비 수출 증감률),수입,(전년동월대비 수입 증감률),무역수지
date,,,,,,,,,,,,,,,,,,,,,
2000-03,152404.4,12.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-06,160404.4,9.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-09,166224.9,9.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-12,172397.6,5.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-03,165662.3,5.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,638,18.8,636,27.9,2
2022-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,579,12.9,603,18.5,-25
2022-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,616,21.4,632,31.9,-16
